In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 30    # steepness factor
num_blocks = 1    # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_exact_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [10:58<00:00,  1.52it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      1.583502               0.0      0.657292
std       2.198429               0.0      1.940810
min       0.000007               0.0      0.091565
25%       0.125648               0.0      0.342334
50%       0.648800               0.0      0.683583
75%       2.039892               0.0      0.794763
max      11.392356               0.0     60.526846
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rnd_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [13:44<00:00,  1.21it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      3.936884          0.030167      0.819774
std       2.994132          0.096639      0.167056
min       0.000821          0.000000      0.294834
25%       1.430063          0.000000      0.696194
50%       3.450611          0.000000      0.796274
75%       5.900992          0.000000      0.905328
max      16.615991          0.485656      1.556693
Number of infeasible solution: 120


## Heuristic - RENS

In [10]:
from src.heuristic import rens

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = rens(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rens_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [21:34<00:00,  1.29s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      1.583312               0.0      1.289498
std       2.198417               0.0      0.241542
min       0.000007               0.0      0.618457
25%       0.125648               0.0      1.137501
50%       0.648800               0.0      1.260613
75%       2.039892               0.0      1.445866
max      11.392356               0.0      2.357556
Number of infeasible solution: 0


## Heuristic - N1

In [12]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:10<00:00, 14.27it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     68.829961               0.0      0.069554
std      42.395181               0.0      0.025004
min       4.334028               0.0      0.058867
25%      32.603227               0.0      0.061585
50%      68.051118               0.0      0.062344
75%      90.896112               0.0      0.063356
max     188.935700               0.0      0.174129
Number of infeasible solution: 0


## Heuristic - N3

In [14]:
model_heur = model.first_solution_heuristic(nodes_limit=3)

In [15]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n3_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:39<00:00,  4.55it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      6.205913               0.0      0.219125
std       8.661385               0.0      0.306805
min       0.000007               0.0      0.060446
25%       0.414089               0.0      0.106020
50%       2.077103               0.0      0.171939
75%       8.261314               0.0      0.249318
max      41.236671               0.0      8.476055
Number of infeasible solution: 0


## Learnable Rounding

In [16]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [17]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# define Rosenbrock objective functions and constraints for both problem types
obj_rel, constrs_rel = nmRosenbrock(["x"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
obj_rnd, constrs_rnd = nmRosenbrock(["x_rnd"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = [smap, rnd]
loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
problem = nm.problem.Problem(components, loss)

In [18]:
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(problem.parameters(), lr=lr)
# create a trainer for the problem
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test, optimizer, 
                            epochs=epochs, patience=patience, warmup=warmup)
# training for the rounding problem
best_model = trainer.train()
# load best model dict
problem.load_state_dict(best_model)

epoch: 0  train_loss: 75.50177764892578
epoch: 1  train_loss: 28.326618194580078
epoch: 2  train_loss: 17.718215942382812
epoch: 3  train_loss: 13.486343383789062
epoch: 4  train_loss: 10.9638032913208
epoch: 5  train_loss: 9.655780792236328
epoch: 6  train_loss: 8.1796293258667
epoch: 7  train_loss: 7.549465656280518
epoch: 8  train_loss: 7.166077136993408
epoch: 9  train_loss: 6.627071857452393
epoch: 10  train_loss: 6.5086774826049805
epoch: 11  train_loss: 6.221748352050781
epoch: 12  train_loss: 5.959054946899414
epoch: 13  train_loss: 6.166877269744873
epoch: 14  train_loss: 5.924166202545166
epoch: 15  train_loss: 5.991180419921875
epoch: 16  train_loss: 5.725766658782959
epoch: 17  train_loss: 5.4449076652526855
epoch: 18  train_loss: 5.6094255447387695
epoch: 19  train_loss: 5.469568729400635
epoch: 20  train_loss: 5.549188613891602
epoch: 21  train_loss: 5.974689960479736
epoch: 22  train_loss: 5.536171913146973
epoch: 23  train_loss: 5.303388595581055
epoch: 24  train_loss: 

<All keys matched successfully>

In [19]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = output["test_x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_nm_e.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 431.57it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      3.889981               0.0      0.002046
std       3.461665               0.0      0.000656
min       0.007193               0.0      0.000998
25%       1.203282               0.0      0.001989
50%       2.713893               0.0      0.002000
75%       5.774637               0.0      0.002241
max      18.585950               0.0      0.005578
Number of infeasible solution: 0
